In [5]:
# ----------------------------------------------------------------------
# 1. Setup and Installation
# ----------------------------------------------------------------------

# This notebook requires the asyncio and websockets libraries.
# Run the following line in a separate cell if you don't have them installed:
!pip install websockets pandas

import asyncio
import websockets
import json
import pandas as pd
from datetime import datetime
import nest_asyncio

# The 'websockets' library is asynchronous. Jupyter notebooks usually run 
# synchronously, so we use 'nest_asyncio' to allow the async loop to run.
nest_asyncio.apply()

# ----------------------------------------------------------------------
# 2. Configuration
# ----------------------------------------------------------------------

# Deriv API uses a public endpoint for market data.
# The 'app_id' is required but can be a default value (like 1089, which is
# commonly used for public testing/data access, or your own registered App ID).
APP_ID = 1089
DERIV_WS_URL = f"wss://ws.derivws.com/websockets/v3?app_id={APP_ID}"

# Define the market symbol you want to track.
# These symbols represent Deriv's MT5 markets, including Synthetic Indices.
# Examples:
# - Volatility 100 Index: "R_100"
# - Volatility 75 Index: "R_75"
# - EUR/USD Forex: "frxEURUSD"
SYMBOL = "R_100" 

# Number of ticks to collect before stopping the script (set to 0 for infinite)
MAX_TICKS = 20

# List to store the received tick data
tick_data_list = []

print(f"Connecting to Deriv API to stream ticks for: {SYMBOL}")
print(f"Will collect up to {MAX_TICKS} ticks.")

# ----------------------------------------------------------------------
# 3. Asynchronous WebSocket Connection and Tick Processing
# ----------------------------------------------------------------------

async def fetch_live_ticks():
    """
    Establishes a WebSocket connection, subscribes to a market, 
    and prints/stores live tick data.
    """
    
    # Use a try/except block to ensure clean closure of the connection
    try:
        # Establish the connection
        async with websockets.connect(DERIV_WS_URL) as websocket:
            print("Connection established successfully.")
            
            # 1. Send the subscription request for live ticks
            subscribe_message = json.dumps({
                "ticks": SYMBOL,
                "subscribe": 1
            })
            await websocket.send(subscribe_message)
            print(f"Subscription request sent for {SYMBOL}...")

            # 2. Main loop to receive and process messages
            while len(tick_data_list) < MAX_TICKS or MAX_TICKS == 0:
                try:
                    # Receive response from the server
                    response = await asyncio.wait_for(websocket.recv(), timeout=10)
                    data = json.loads(response)

                    # Check for errors
                    if 'error' in data:
                        print(f"API Error: {data['error'].get('message', 'Unknown error')}")
                        break

                    # Process tick data
                    if 'tick' in data:
                        tick = data['tick']
                        
                        # Extract and format the timestamp
                        timestamp_sec = tick['epoch']
                        dt_object = datetime.fromtimestamp(timestamp_sec)

                        # Extract the price (quote)
                        price = float(tick['quote'])

                        # Store the data
                        tick_data_list.append({
                            'time': dt_object.strftime('%Y-%m-%d %H:%M:%S.%f'),
                            'symbol': tick['symbol'],
                            'price': price,
                            'epoch': timestamp_sec
                        })

                        # Print the latest tick
                        print(f"TICK {len(tick_data_list):02d}/{MAX_TICKS if MAX_TICKS > 0 else '∞'}: "
                              f"[{dt_object.time()}] {tick['symbol']} @ {price:0.5f}")

                        # Stop if maximum ticks reached
                        if MAX_TICKS > 0 and len(tick_data_list) >= MAX_TICKS:
                            # 3. Unsubscribe before closing (good practice)
                            unsub_id = data.get('tick', {}).get('id')
                            if unsub_id:
                                forget_message = json.dumps({"forget": unsub_id})
                                await websocket.send(forget_message)
                                print(f"\nUnsubscribed from tick stream ID: {unsub_id}")
                            break

                    # Handle initial confirmation message (not a tick)
                    elif 'msg_type' in data and data['msg_type'] == 'tick':
                        # This is usually the first historical tick or stream start confirmation
                        continue 
                        
                except asyncio.TimeoutError:
                    # Timeout after 10 seconds without data; usually indicates a dead connection
                    print("Connection timed out. Reconnecting...")
                    break 
                
                except websockets.exceptions.ConnectionClosedOK:
                    print("Connection closed by server.")
                    break
                
    except ConnectionRefusedError:
        print("FATAL ERROR: Could not connect to the Deriv API URL.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# ----------------------------------------------------------------------
# 4. Execute the Script
# ----------------------------------------------------------------------

# The loop.run_until_complete() is used in a Jupyter/IPython environment 
# to run the asynchronous function synchronously.
asyncio.get_event_loop().run_until_complete(fetch_live_ticks())

# ----------------------------------------------------------------------
# 5. Display the Collected Data
# ----------------------------------------------------------------------

if tick_data_list:
    df = pd.DataFrame(tick_data_list)
    df['time'] = pd.to_datetime(df['time'])
    df = df.set_index('time')
    
    print("\n--- Collected Ticks DataFrame ---")
    print(df.drop(columns=['epoch']))
    
    # Optional: Plot the price movement (requires matplotlib to be installed: pip install matplotlib)
    # import matplotlib.pyplot as plt
    # plt.figure(figsize=(10, 6))
    # df['price'].plot(title=f'Live Price Ticks for {SYMBOL}')
    # plt.ylabel('Price (Quote)')
    # plt.grid(True)
    # plt.show()
else:
    print("\nNo tick data was collected.")

Connecting to Deriv API to stream ticks for: R_100
Will collect up to 20 ticks.
Connection established successfully.
Subscription request sent for R_100...
TICK 01/20: [21:32:20] R_100 @ 865.04000
TICK 02/20: [21:32:22] R_100 @ 865.14000
TICK 03/20: [21:32:24] R_100 @ 864.99000
TICK 04/20: [21:32:26] R_100 @ 865.11000
TICK 05/20: [21:32:28] R_100 @ 865.19000
TICK 06/20: [21:32:30] R_100 @ 865.63000
TICK 07/20: [21:32:32] R_100 @ 865.88000
TICK 08/20: [21:32:34] R_100 @ 865.81000
TICK 09/20: [21:32:36] R_100 @ 865.70000
TICK 10/20: [21:32:38] R_100 @ 865.72000
TICK 11/20: [21:32:40] R_100 @ 865.57000
TICK 12/20: [21:32:42] R_100 @ 865.66000
TICK 13/20: [21:32:44] R_100 @ 865.57000
Connection timed out. Reconnecting...

--- Collected Ticks DataFrame ---
                    symbol   price
time                              
2025-11-20 21:32:20  R_100  865.04
2025-11-20 21:32:22  R_100  865.14
2025-11-20 21:32:24  R_100  864.99
2025-11-20 21:32:26  R_100  865.11
2025-11-20 21:32:28  R_100  

In [ ]:
# ----------------------------------------------------------------------
# 1. Setup and Installation
# ----------------------------------------------------------------------

# This notebook requires the asyncio and websockets libraries.
# Run the following line in a separate cell if you don't have them installed:
# !pip install websockets pandas

import asyncio
import websockets
import json
import pandas as pd
from datetime import datetime
import nest_asyncio
import random # Used for generating random client_id for demonstration

# The 'websockets' library is asynchronous. Jupyter notebooks usually run 
# synchronously, so we use 'nest_asyncio' to allow the async loop to run.
nest_asyncio.apply()

# ----------------------------------------------------------------------
# 2. Configuration & Authentication
# ----------------------------------------------------------------------

# *** MANDATORY: REPLACE WITH YOUR ACTUAL API TOKEN ***
# You can generate a token in your Deriv account settings (Security > API Token).
# Ensure the token has 'Read' and 'Trade' permissions.
API_TOKEN = 'YOUR_DERIV_API_TOKEN_HERE' 

APP_ID = 1089 # Default public App ID
DERIV_WS_URL = f"wss://ws.derivws.com/websockets/v3?app_id={APP_ID}"

# Define the market symbols you want to track.
# We will track a Synthetic Index and a Forex pair.
SYMBOLS = ["R_100", "frxEURUSD"] 

# Number of ticks to collect before stopping the script (set to 0 for infinite)
MAX_TICKS = 50 

# Data Storage
tick_data_list = []
# Dictionary to store the latest tick quote for simple strategy implementation
latest_prices = {symbol: None for symbol in SYMBOLS} 

# ----------------------------------------------------------------------
# 3. Trading Function (Sends Proposal and Buy requests)
# ----------------------------------------------------------------------

async def place_trade(websocket, symbol, contract_type, amount):
    """
    Submits a proposal request, waits for the price, and then buys the contract.
    This is a basic 1-tick contract for demonstration purposes.
    """
    client_req_id = random.randint(1000, 9999) # Unique ID for this request

    print(f"\n--- ATTEMPTING TRADE on {symbol} ({contract_type}) ---")

    # 1. Send Proposal: Get the price for the desired contract
    proposal_message = json.dumps({
        "proposal": 1,
        "subscribe": 0, # Only need a single price, not a stream
        "amount": amount,
        "basis": "stake",
        "contract_type": contract_type,
        "currency": "USD",
        "duration": 5, # 5 ticks duration
        "duration_unit": "t",
        "symbol": symbol,
        "req_id": client_req_id
    })
    await websocket.send(proposal_message)
    print("Proposal request sent. Waiting for price...")
    
    # Wait for the proposal response
    proposal_response = await asyncio.wait_for(websocket.recv(), timeout=5)
    proposal_data = json.loads(proposal_response)
    
    if 'error' in proposal_data:
        print(f"!!! Proposal Error: {proposal_data['error'].get('message')}")
        return

    proposal_id = proposal_data.get('proposal', {}).get('id')
    price = proposal_data.get('proposal', {}).get('ask_price')
    
    if not proposal_id or not price:
        print("!!! Could not get a valid proposal ID or Price.")
        return

    print(f"Proposal received: ID={proposal_id}, Ask Price={price}")

    # 2. Send Buy Request
    buy_message = json.dumps({
        "buy": proposal_id,
        "price": price, # Use the proposed price
        "req_id": client_req_id + 1 # Use a different request ID for the buy
    })
    await websocket.send(buy_message)
    
    # Wait for the buy confirmation response
    buy_response = await asyncio.wait_for(websocket.recv(), timeout=5)
    buy_data = json.loads(buy_response)

    if 'error' in buy_data:
        print(f"!!! Buy Error: {buy_data['error'].get('message')}")
    else:
        contract_info = buy_data.get('buy', {})
        print(f"*** TRADE SUCCESSFUL ***")
        print(f"   Contract ID: {contract_info.get('contract_id')}")
        print(f"   Entry Spot: {contract_info.get('buy_price')}")
        print(f"   Details: {buy_data.get('purchase', {}).get('longcode')}")
        print("---------------------------------------")


# ----------------------------------------------------------------------
# 4. Asynchronous WebSocket Connection and Tick Processing
# ----------------------------------------------------------------------

async def fetch_live_ticks_and_trade():
    """
    Handles connection, authorization, multi-symbol subscription, 
    and rudimentary trading logic.
    """
    
    if API_TOKEN == 'YOUR_DERIV_API_TOKEN_HERE':
        print("\n--- WARNING ---")
        print("Please replace 'YOUR_DERIV_API_TOKEN_HERE' with your actual API token.")
        print("The script will only fetch public ticks and cannot place trades.")
        print("-------------\n")

    try:
        async with websockets.connect(DERIV_WS_URL) as websocket:
            print("Connection established successfully.")
            
            # 1. Authorization (Required for trading)
            if API_TOKEN and API_TOKEN != 'YOUR_DERIV_API_TOKEN_HERE':
                auth_message = json.dumps({"authorize": API_TOKEN})
                await websocket.send(auth_message)
                auth_response = await asyncio.wait_for(websocket.recv(), timeout=5)
                auth_data = json.loads(auth_response)

                if 'error' in auth_data:
                    print(f"!!! Authorization FAILED: {auth_data['error'].get('message')}")
                    return # Exit if auth fails
                
                account_id = auth_data.get('authorize', {}).get('loginid')
                currency = auth_data.get('authorize', {}).get('currency')
                print(f"Authorization successful. Account ID: {account_id} ({currency})")
            
            # 2. Subscribe to Ticks for All Symbols
            for symbol in SYMBOLS:
                subscribe_message = json.dumps({"ticks": symbol, "subscribe": 1})
                await websocket.send(subscribe_message)
                print(f"Subscription request sent for {symbol}...")

            # 3. Main loop to receive and process messages
            trades_executed = 0
            while len(tick_data_list) < MAX_TICKS or MAX_TICKS == 0:
                try:
                    response = await asyncio.wait_for(websocket.recv(), timeout=15)
                    data = json.loads(response)

                    if 'error' in data:
                        print(f"API Error: {data['error'].get('message', 'Unknown error')}")
                        continue
                        
                    # Process tick data
                    if 'tick' in data:
                        tick = data['tick']
                        symbol = tick['symbol']
                        
                        # Extract and format the timestamp and price
                        dt_object = datetime.fromtimestamp(tick['epoch'])
                        price = float(tick['quote'])
                        
                        # Update latest price
                        latest_prices[symbol] = price

                        # Store data
                        tick_data_list.append({
                            'time': dt_object.strftime('%Y-%m-%d %H:%M:%S.%f'),
                            'symbol': symbol,
                            'price': price,
                            'epoch': tick['epoch']
                        })

                        # Print the latest tick
                        print(f"TICK {len(tick_data_list):02d}/{MAX_TICKS if MAX_TICKS > 0 else '∞'}: "
                              f"[{dt_object.time()}] {symbol} @ {price:0.5f}")

                        # --- PLACEHOLDER TRADING STRATEGY ---
                        # Execute a trade on R_100 every 10 ticks for demonstration
                        if symbol == "R_100" and trades_executed < 2: # Limit trades to 2
                            if len(tick_data_list) % 10 == 0:
                                # Simple "Rise" (CALL) or "Fall" (PUT) prediction based on the tick number
                                contract = "CALL" if len(tick_data_list) // 10 % 2 == 0 else "PUT"
                                print(f"** TRADING SIGNAL: {contract} for {symbol} triggered! **")
                                # Run the trading logic as a separate task to not block the tick stream
                                await place_trade(websocket, symbol, contract, amount=1.0) # Stake $1
                                trades_executed += 1


                        # Stop if maximum ticks reached
                        if MAX_TICKS > 0 and len(tick_data_list) >= MAX_TICKS:
                            # 4. Unsubscribe from all streams
                            for subscription_id in data.get('tick', {}).get('id'):
                                forget_message = json.dumps({"forget": subscription_id})
                                await websocket.send(forget_message)
                            print("\nUnsubscribed from all tick streams.")
                            break

                    # Handle initial confirmation messages (which don't contain 'tick')
                    elif 'msg_type' in data:
                        if data['msg_type'] in ['authorize', 'ticks', 'proposal', 'buy', 'forget']:
                             # Print authorization results or other API responses
                            if data['msg_type'] != 'tick': # Ticks are handled above
                                # print(f"Received {data['msg_type']} response.") 
                                pass

                    
                except asyncio.TimeoutError:
                    print("Connection timed out (no data for 15s). Exiting.")
                    break 
                
                except websockets.exceptions.ConnectionClosedOK:
                    print("WebSocket closed cleanly.")
                    break
                
    except ConnectionRefusedError:
        print("FATAL ERROR: Could not connect to the Deriv API URL.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# ----------------------------------------------------------------------
# 5. Execute the Script
# ----------------------------------------------------------------------

print(f"Connecting to Deriv API to stream ticks for: {', '.join(SYMBOLS)}")
print(f"Will collect up to {MAX_TICKS} ticks and execute up to 2 trades on R_100.")

# Run the asynchronous function
asyncio.get_event_loop().run_until_complete(fetch_live_ticks_and_trade())

# ----------------------------------------------------------------------
# 6. Display the Collected Data
# ----------------------------------------------------------------------

if tick_data_list:
    df = pd.DataFrame(tick_data_list)
    df['time'] = pd.to_datetime(df['time'])
    df = df.set_index('time')
    
    print("\n--- Collected Ticks DataFrame ---")
    print(df.drop(columns=['epoch']))
    
else:
    print("\nNo tick data was collected.")

In [10]:
# ----------------------------------------------------------------------
# 1. Setup and Installation
# ----------------------------------------------------------------------

# This notebook requires the MetaTrader5 library.
# Run the following line in a separate cell if you don't have it installed:
# !pip install MetaTrader5 pandas

import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
import time
import random

# ----------------------------------------------------------------------
# 2. Configuration & MT5 Account Details
# ----------------------------------------------------------------------

# *** MANDATORY: REPLACE WITH YOUR ACTUAL MT5 LOGIN DETAILS ***
# These details are from your Deriv MT5 account (Demo or Real).
MT5_LOGIN = 40866995  # Your MT5 Login ID
MT5_PASSWORD = "NaOmI5n424576$" 
MT5_SERVER = "Deriv-Demo"  # Use "Deriv-Demo" or "Deriv-Server"

# Define the market symbols you want to track and trade.
# Ensure these symbols are visible in your MT5 Market Watch.
SYMBOLS = ["Volatility 100 Index", "EURUSD"] 

# Trading Parameters
TRADE_VOLUME = 0.01  # Standard lot size for Synthetic Indices
MAX_TICKS = 50       # Number of ticks to fetch before stopping

# Data Storage
tick_data_list = []

# ----------------------------------------------------------------------
# 3. Trading Function (Sends Order to MT5)
# ----------------------------------------------------------------------

def place_trade(symbol, trade_type, volume):
    """
    Submits an order to the connected MT5 terminal.
    
    :param symbol: Trading symbol (e.g., "Volatility 100 Index")
    :param trade_type: mt5.BUY or mt5.SELL
    :param volume: Lot size
    :return: True if the order was sent successfully, False otherwise.
    """
    
    # 1. Get the latest price to set the order
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f"!!! Error: Symbol {symbol} not found or MT5 not connected.")
        return False
        
    point = symbol_info.point
    
    # Determine price and type
    if trade_type == mt5.ORDER_TYPE_BUY:
        price = mt5.symbol_info_tick(symbol).ask
        action = mt5.TRADE_REQUEST_ACTIONS_DEAL
        order_type = mt5.ORDER_TYPE_BUY
    else: # SELL
        price = mt5.symbol_info_tick(symbol).bid
        action = mt5.TRADE_REQUEST_ACTIONS_DEAL
        order_type = mt5.ORDER_TYPE_SELL

    # 2. Prepare the trade request
    request = {
        "action": action,
        "symbol": symbol,
        "volume": volume,
        "type": order_type,
        "price": price,
        "deviation": 20, # Acceptable price deviation from current price (in points)
        "magic": 202409, # Custom ID for the bot (optional)
        "comment": "Python Trade Bot",
        "type_time": mt5.ORDER_TIME_GTC, # Good 'til cancelled
        "type_filling": mt5.ORDER_FILLING_FOK, # Fill or Kill
    }
    
    # 3. Send the order
    result = mt5.order_send(request)
    
    print(f"\n--- TRADING SIGNAL for {symbol} ({'BUY' if trade_type == mt5.ORDER_TYPE_BUY else 'SELL'}) ---")
    
    # 4. Check the result
    if result.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"!!! Order Send Failed. Code: {result.retcode}")
        print(f"Result Comment: {result.comment}")
        return False
    else:
        print(f"*** TRADE EXECUTED ***")
        print(f"   Order: {result.order}")
        print(f"   Ticket: {result.deal}")
        print(f"   Price: {result.price}")
        print("---------------------------------------")
        return True

# ----------------------------------------------------------------------
# 4. Main Script Execution
# ----------------------------------------------------------------------

if __name__ == "__main__":
    
    # 1. Initialize MT5 Connection
    if not mt5.initialize():
        print("MT5 Initialization FAILED. Check installation and environment.")
        # Attempt to initialize with credentials
        #if not mt5.initialize(login=MT5_LOGIN, password=MT5_PASSWORD, server=MT5_SERVER):
        if not mt5.initialize():
             print(f"!!! FAILED TO CONNECT AND LOGIN: {mt5.last_error()}")
             exit()
    
    print(f"MT5 initialized and connected. Server: {mt5.terminal_info().name}")
    
    if mt5.login(MT5_LOGIN, password=MT5_PASSWORD, server=MT5_SERVER):
        print(f"Login successful for account: {MT5_LOGIN}")
    else:
        print(f"!!! Login FAILED: {mt5.last_error()}")
        mt5.shutdown()
        exit()

    # 2. Main Loop for Ticks and Trading
    trades_executed = 0
    start_time = time.time()
    
    print(f"\nStarting live tick fetch for {', '.join(SYMBOLS)}. Max Ticks: {MAX_TICKS}")

    while len(tick_data_list) < MAX_TICKS:
        
        for symbol in SYMBOLS:
            # Get the latest tick information
            tick = mt5.symbol_info_tick(symbol)
            
            if tick and tick.bid > 0 and tick.time != tick_data_list[-1]['epoch'] if tick_data_list else True:
                
                # Extract and format data
                dt_object = datetime.fromtimestamp(tick.time)
                
                # Check for duplicate ticks (MT5 can sometimes send the same tick multiple times)
                if tick_data_list and tick_data_list[-1]['epoch'] == tick.time:
                    continue 

                # Store data
                tick_data_list.append({
                    'time': dt_object.strftime('%Y-%m-%d %H:%M:%S.%f'),
                    'symbol': symbol,
                    'bid': tick.bid,
                    'ask': tick.ask,
                    'epoch': tick.time
                })

                # Print the latest tick
                print(f"TICK {len(tick_data_list):02d}/{MAX_TICKS}: "
                      f"[{dt_object.time()}] {symbol} | BID: {tick.bid:0.5f} | ASK: {tick.ask:0.5f}")

                # --- PLACEHOLDER TRADING STRATEGY ---
                # Execute a trade on Volatility 100 Index every 10 ticks (total 2 trades)
                if symbol == "Volatility 100 Index" and trades_executed < 2:
                    if len(tick_data_list) % 10 == 0:
                        
                        # Simple random trade direction for demonstration
                        trade_direction = random.choice([mt5.ORDER_TYPE_BUY, mt5.ORDER_TYPE_SELL])
                        
                        print(f"** TRADING SIGNAL: {'BUY' if trade_direction == mt5.ORDER_TYPE_BUY else 'SELL'} for {symbol} triggered! **")
                        
                        if place_trade(symbol, trade_direction, TRADE_VOLUME):
                            trades_executed += 1

        # Small delay to prevent overwhelming the MT5 terminal
        time.sleep(0.1) 
        
    # 3. Clean up
    print("\nMT5 connection shutting down.")
    mt5.shutdown()

    # 4. Display the Collected Data
    if tick_data_list:
        df = pd.DataFrame(tick_data_list)
        df['time'] = pd.to_datetime(df['time'])
        df = df.set_index('time')
        
        print("\n--- Collected Ticks DataFrame ---")
        print(df.drop(columns=['epoch']))
        
    else:
        print("\nNo tick data was collected.")

MT5 initialized and connected. Server: MetaTrader 5
Login successful for account: 40866995

Starting live tick fetch for Volatility 100 Index, EURUSD. Max Ticks: 50
TICK 01/50: [22:06:12] Volatility 100 Index | BID: 867.24000 | ASK: 867.49000
TICK 02/50: [22:06:07] EURUSD | BID: 1.15245 | ASK: 1.15255
TICK 03/50: [22:06:12] Volatility 100 Index | BID: 867.24000 | ASK: 867.49000
TICK 04/50: [22:06:07] EURUSD | BID: 1.15245 | ASK: 1.15255
TICK 05/50: [22:06:12] Volatility 100 Index | BID: 867.24000 | ASK: 867.49000
TICK 06/50: [22:06:07] EURUSD | BID: 1.15245 | ASK: 1.15255
TICK 07/50: [22:06:12] Volatility 100 Index | BID: 867.24000 | ASK: 867.49000
TICK 08/50: [22:06:07] EURUSD | BID: 1.15245 | ASK: 1.15255
TICK 09/50: [22:06:12] Volatility 100 Index | BID: 867.24000 | ASK: 867.49000
TICK 10/50: [22:06:07] EURUSD | BID: 1.15245 | ASK: 1.15255
TICK 11/50: [22:06:12] Volatility 100 Index | BID: 867.24000 | ASK: 867.49000
TICK 12/50: [22:06:07] EURUSD | BID: 1.15245 | ASK: 1.15255
TICK 13

In [11]:
# ----------------------------------------------------------------------
# 1. Setup and Installation
# ----------------------------------------------------------------------

# This notebook requires the MetaTrader5 library.
# Run the following line in a separate cell if you don't have it installed:
# !pip install MetaTrader5 pandas

import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
import time
import random
# Removed 'import os' as we no longer use environment variables

# ----------------------------------------------------------------------
# 2. Configuration & Symbols
# ----------------------------------------------------------------------

# *** IMPORTANT: This script relies on your MetaTrader 5 terminal being ***
# *** running and already logged into your Deriv account on this machine. ***

# Define the market symbols you want to track and trade.
# Ensure these symbols are visible in your MT5 Market Watch.
SYMBOLS = ["Volatility 100 Index", "EURUSD"] 

# Trading Parameters
TRADE_VOLUME = 0.01  # Standard lot size for Synthetic Indices (adjust as needed)
MAX_TICKS = 50       # Number of ticks to fetch before stopping

# Data Storage
tick_data_list = []

# Global placeholder for the logged-in account ID
MT5_LOGIN = None 

# ----------------------------------------------------------------------
# 3. Trading Function (Sends Order to MT5)
# ----------------------------------------------------------------------

def place_trade(symbol, trade_type, volume):
    """
    Submits an order to the connected MT5 terminal.
    
    :param symbol: Trading symbol (e.g., "Volatility 100 Index")
    :param trade_type: mt5.BUY or mt5.SELL
    :param volume: Lot size
    :return: True if the order was sent successfully, False otherwise.
    """
    
    # 1. Get the latest price to set the order
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f"!!! Error: Symbol {symbol} not found or MT5 not connected.")
        return False
        
    # Determine price and type
    if trade_type == mt5.ORDER_TYPE_BUY:
        price = mt5.symbol_info_tick(symbol).ask
        action = mt5.TRADE_REQUEST_ACTIONS_DEAL
        order_type = mt5.ORDER_TYPE_BUY
    else: # SELL
        price = mt5.symbol_info_tick(symbol).bid
        action = mt5.TRADE_REQUEST_ACTIONS_DEAL
        order_type = mt5.ORDER_TYPE_SELL
        
    # Check if a valid price was obtained
    if price is None or price <= 0:
        print(f"!!! Error: Could not get a valid price for {symbol}.")
        return False

    # 2. Prepare the trade request
    request = {
        "action": action,
        "symbol": symbol,
        "volume": volume,
        "type": order_type,
        "price": price,
        "deviation": 20, # Acceptable price deviation from current price (in points)
        "magic": 202409, # Custom ID for the bot (optional)
        "comment": "Auto-Connect Bot",
        "type_time": mt5.ORDER_TIME_GTC, # Good 'til cancelled
        "type_filling": mt5.ORDER_FILLING_FOK, # Fill or Kill
    }
    
    # 3. Send the order
    result = mt5.order_send(request)
    
    print(f"\n--- TRADING SIGNAL for {symbol} ({'BUY' if trade_type == mt5.ORDER_TYPE_BUY else 'SELL'}) ---")
    
    # 4. Check the result
    if result.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"!!! Order Send Failed. Code: {result.retcode}")
        print(f"Result Comment: {result.comment}")
        return False
    else:
        print(f"*** TRADE EXECUTED ***")
        print(f"   Order: {result.order}")
        print(f"   Ticket: {result.deal}")
        print(f"   Price: {result.price}")
        print("---------------------------------------")
        return True

# ----------------------------------------------------------------------
# 4. Main Script Execution
# ----------------------------------------------------------------------

if __name__ == "__main__":
    
    # 1. Initialize MT5 Connection (Auto-Connect to running terminal)
    if not mt5.initialize():
        print(f"!!! MT5 Initialization FAILED. Check if terminal is running: {mt5.last_error()}")
        exit()
    
    # 2. Check Login Status
    account_info = mt5.account_info()
    if account_info is None:
        print("!!! FAILED TO CONNECT to a logged-in account.")
        print("!!! Please ensure your MT5 terminal is open and logged into your Deriv account.")
        mt5.shutdown()
        exit()

    # Login is successful via the running terminal
    global MT5_LOGIN
    MT5_LOGIN = account_info.login
    
    print(f"MT5 initialized and connected automatically.")
    print(f"Logged in to account: {MT5_LOGIN} | Server: {mt5.terminal_info().name}")

    # 3. Main Loop for Ticks and Trading
    trades_executed = 0
    start_time = time.time()
    
    print(f"\nStarting live tick fetch for {', '.join(SYMBOLS)}. Max Ticks: {MAX_TICKS}")

    while len(tick_data_list) < MAX_TICKS:
        
        for symbol in SYMBOLS:
            # Get the latest tick information
            tick = mt5.symbol_info_tick(symbol)
            
            # Check for tick validity and uniqueness
            is_new_tick = True
            if tick and tick.bid > 0 and tick_data_list and tick.time == tick_data_list[-1]['epoch']:
                is_new_tick = False
                
            if tick and tick.bid > 0 and is_new_tick:
                
                # Extract and format data
                dt_object = datetime.fromtimestamp(tick.time)
                
                # Store data
                tick_data_list.append({
                    'time': dt_object.strftime('%Y-%m-%d %H:%M:%S.%f'),
                    'symbol': symbol,
                    'bid': tick.bid,
                    'ask': tick.ask,
                    'epoch': tick.time
                })

                # Print the latest tick
                print(f"TICK {len(tick_data_list):02d}/{MAX_TICKS}: "
                      f"[{dt_object.time()}] {symbol} | BID: {tick.bid:0.5f} | ASK: {tick.ask:0.5f}")

                # --- PLACEHOLDER TRADING STRATEGY ---
                # Execute a trade on Volatility 100 Index every 10 ticks (total 2 trades)
                if symbol == "Volatility 100 Index" and trades_executed < 2:
                    if len(tick_data_list) % 10 == 0:
                        
                        # Simple random trade direction for demonstration
                        trade_direction = random.choice([mt5.ORDER_TYPE_BUY, mt5.ORDER_TYPE_SELL])
                        
                        print(f"** TRADING SIGNAL: {'BUY' if trade_direction == mt5.ORDER_TYPE_BUY else 'SELL'} for {symbol} triggered! **")
                        
                        if place_trade(symbol, trade_direction, TRADE_VOLUME):
                            trades_executed += 1

        # Small delay to prevent overwhelming the MT5 terminal
        time.sleep(0.1) 
        
    # 4. Clean up
    print("\nMT5 connection shutting down.")
    mt5.shutdown()

    # 5. Display the Collected Data
    if tick_data_list:
        df = pd.DataFrame(tick_data_list)
        df['time'] = pd.to_datetime(df['time'])
        df = df.set_index('time')
        
        print("\n--- Collected Ticks DataFrame ---")
        print(df.drop(columns=['epoch']))
        
    else:
        print("\nNo tick data was collected.")

MT5 initialized and connected automatically.
Logged in to account: 40866995 | Server: MetaTrader 5

Starting live tick fetch for Volatility 100 Index, EURUSD. Max Ticks: 50
TICK 01/50: [22:07:48] Volatility 100 Index | BID: 867.55000 | ASK: 867.80000
TICK 02/50: [22:07:38] EURUSD | BID: 1.15251 | ASK: 1.15261
TICK 03/50: [22:07:48] Volatility 100 Index | BID: 867.55000 | ASK: 867.80000
TICK 04/50: [22:07:38] EURUSD | BID: 1.15251 | ASK: 1.15261
TICK 05/50: [22:07:48] Volatility 100 Index | BID: 867.55000 | ASK: 867.80000
TICK 06/50: [22:07:38] EURUSD | BID: 1.15251 | ASK: 1.15261
TICK 07/50: [22:07:48] Volatility 100 Index | BID: 867.55000 | ASK: 867.80000
TICK 08/50: [22:07:38] EURUSD | BID: 1.15251 | ASK: 1.15261
TICK 09/50: [22:07:50] Volatility 100 Index | BID: 867.46000 | ASK: 867.71000
TICK 10/50: [22:07:38] EURUSD | BID: 1.15251 | ASK: 1.15261
TICK 11/50: [22:07:50] Volatility 100 Index | BID: 867.46000 | ASK: 867.71000
TICK 12/50: [22:07:38] EURUSD | BID: 1.15251 | ASK: 1.15261

In [21]:
# ----------------------------------------------------------------------
# 1. Setup and Installation
# ----------------------------------------------------------------------

# This notebook requires the MetaTrader5 library.
# Run the following line in a separate cell if you don't have it installed:
# !pip install MetaTrader5 pandas

import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
import time
import random
# Removed 'import os' as we no longer use environment variables

# ----------------------------------------------------------------------
# 2. Configuration & Symbols
# ----------------------------------------------------------------------

# *** IMPORTANT: This script relies on your MetaTrader 5 terminal being ***
# *** running and already logged into your Deriv account on this machine. ***

# Define the market symbols you want to track and trade.
# Ensure these symbols are visible in your MT5 Market Watch.
SYMBOLS = ["Volatility 100 Index"] 

# Trading Parameters
TRADE_VOLUME = 1  # Standard lot size for Synthetic Indices (adjust as needed)
MAX_TICKS = 60       # Number of ticks to fetch before stopping (Increased for more trades)

# Data Storage
tick_data_list = []

# Global placeholder for the logged-in account ID
MT5_LOGIN = None 

# ----------------------------------------------------------------------
# 3. Trading Function (Sends Order to MT5)
# ----------------------------------------------------------------------

def place_trade(symbol, trade_type, volume):
    """
    Submits an order to the connected MT5 terminal.
    
    :param symbol: Trading symbol (e.g., "Volatility 100 Index")
    :param trade_type: mt5.BUY or mt5.SELL
    :param volume: Lot size
    :return: True if the order was sent successfully, False otherwise.
    """
    
    # 1. Get the latest price to set the order
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f"!!! Error: Symbol {symbol} not found or MT5 not connected.")
        return False
        
    # Determine price and type
    if trade_type == mt5.ORDER_TYPE_BUY:
        price = mt5.symbol_info_tick(symbol).ask
        action = mt5.TRADE_ACTION_DEAL
        order_type = mt5.ORDER_TYPE_BUY
    else: # SELL
        price = mt5.symbol_info_tick(symbol).bid
        action = mt5.TRADE_ACTION_DEAL
        order_type = mt5.ORDER_TYPE_SELL
        
    # Check if a valid price was obtained
    if price is None or price <= 0:
        print(f"!!! Error: Could not get a valid price for {symbol}.")
        return False

    # 2. Prepare the trade request
    request = {
        "action": action,
        "symbol": symbol,
        "volume": volume,
        "type": order_type,
        "price": price,
        "deviation": 20, # Acceptable price deviation from current price (in points)
        "magic": 202409, # Custom ID for the bot (optional)
        "comment": "Auto-Connect Bot",
        "type_time": mt5.ORDER_TIME_GTC, # Good 'til cancelled
        "type_filling": mt5.ORDER_FILLING_FOK, # Fill or Kill
    }
    
    # 3. Send the order
    result = mt5.order_send(request)
    
    print(f"\n--- TRADING SIGNAL for {symbol} ({'BUY' if trade_type == mt5.ORDER_TYPE_BUY else 'SELL'}) ---")
    
    # 4. Check the result
    if result.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"!!! Order Send Failed. Code: {result.retcode}")
        print(f"Result Comment: {result.comment}")
        return False
    else:
        print(f"*** TRADE EXECUTED ***")
        print(f"   Order: {result.order}")
        print(f"   Ticket: {result.deal}")
        print(f"   Price: {result.price}")
        print("---------------------------------------")
        return True

# ----------------------------------------------------------------------
# 4. Main Script Execution
# ----------------------------------------------------------------------

if __name__ == "__main__":
    
    # 1. Initialize MT5 Connection (Auto-Connect to running terminal)
    if not mt5.initialize():
        print(f"!!! MT5 Initialization FAILED. Check if terminal is running: {mt5.last_error()}")
        exit()
    
    # 2. Check Login Status
    account_info = mt5.account_info()
    if account_info is None:
        print("!!! FAILED TO CONNECT to a logged-in account.")
        print("!!! Please ensure your MT5 terminal is open and logged into your Deriv account.")
        mt5.shutdown()
        exit()

    # Login is successful via the running terminal
    global MT5_LOGIN
    MT5_LOGIN = account_info.login
    
    print(f"MT5 initialized and connected automatically.")
    print(f"Logged in to account: {MT5_LOGIN} | Server: {mt5.terminal_info().name}")

    # 3. Main Loop for Ticks and Trading
    trades_executed = 0
    start_time = time.time()
    
    print(f"\nStarting live tick fetch for {', '.join(SYMBOLS)}. Max Ticks: {MAX_TICKS}")

    while len(tick_data_list) < MAX_TICKS:
        
        for symbol in SYMBOLS:
            # Get the latest tick information
            tick = mt5.symbol_info_tick(symbol)
            
            # Check for tick validity and uniqueness
            is_new_tick = True
            if tick and tick.bid > 0 and tick_data_list and tick.time == tick_data_list[-1]['epoch']:
                is_new_tick = False
                
            if tick and tick.bid > 0 and is_new_tick:
                
                # Extract and format data
                dt_object = datetime.fromtimestamp(tick.time)
                
                # Store data
                tick_data_list.append({
                    'time': dt_object.strftime('%Y-%m-%d %H:%M:%S.%f'),
                    'symbol': symbol,
                    'bid': tick.bid,
                    'ask': tick.ask,
                    'epoch': tick.time
                })

                # Print the latest tick
                print(f"TICK {len(tick_data_list):02d}/{MAX_TICKS}: "
                      f"[{dt_object.time()}] {symbol} | BID: {tick.bid:0.5f} | ASK: {tick.ask:0.5f}")

                # --- RANDOM TRADING STRATEGY (DEMO) ---
                # Execute a random trade on Volatility 100 Index every 5 ticks (max 4 trades)
                if symbol == "Volatility 100 Index" and trades_executed < 4:
                    if len(tick_data_list) % 5 == 0:
                        
                        # Determine trade direction randomly
                        trade_direction = random.choice([mt5.ORDER_TYPE_BUY, mt5.ORDER_TYPE_SELL])
                        
                        print(f"** RANDOM TRADE TRIGGERED: {'BUY' if trade_direction == mt5.ORDER_TYPE_BUY else 'SELL'} for {symbol} **")
                        
                        if place_trade(symbol, trade_direction, TRADE_VOLUME):
                            trades_executed += 1

        # Small delay to prevent overwhelming the MT5 terminal
        time.sleep(0.1) 
        
    # 4. Clean up
    print("\nMT5 connection shutting down.")
    mt5.shutdown()

    # 5. Display the Collected Data
    if tick_data_list:
        df = pd.DataFrame(tick_data_list)
        df['time'] = pd.to_datetime(df['time'])
        df = df.set_index('time')
        
        print("\n--- Collected Ticks DataFrame ---")
        print(df.drop(columns=['epoch']))
        
    else:
        print("\nNo tick data was collected.")

MT5 initialized and connected automatically.
Logged in to account: 40866995 | Server: MetaTrader 5

Starting live tick fetch for Volatility 100 Index. Max Ticks: 60
TICK 01/60: [22:27:06] Volatility 100 Index | BID: 871.12000 | ASK: 871.37000
TICK 02/60: [22:27:08] Volatility 100 Index | BID: 871.08000 | ASK: 871.33000
TICK 03/60: [22:27:10] Volatility 100 Index | BID: 871.12000 | ASK: 871.37000
TICK 04/60: [22:27:12] Volatility 100 Index | BID: 871.14000 | ASK: 871.39000
TICK 05/60: [22:27:14] Volatility 100 Index | BID: 871.25000 | ASK: 871.50000
** RANDOM TRADE TRIGGERED: BUY for Volatility 100 Index **

--- TRADING SIGNAL for Volatility 100 Index (BUY) ---


AttributeError: 'NoneType' object has no attribute 'retcode'

In [25]:
# This version can place trades
# ----------------------------------------------------------------------
# 1. Setup and Installation
# ----------------------------------------------------------------------

# This notebook requires the MetaTrader5 library.
# Run the following line in a separate cell if you don't have it installed:
# !pip install MetaTrader5 pandas

import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
import time
import random
# Removed 'import os' as we no longer use environment variables

# ----------------------------------------------------------------------
# 2. Configuration & Symbols
# ----------------------------------------------------------------------

# *** IMPORTANT: This script relies on your MetaTrader 5 terminal being ***
# *** running and already logged into your Deriv account on this machine. ***

# Define the market symbols you want to track and trade.
# Ensure these symbols are visible in your MT5 Market Watch.
SYMBOLS = ["Volatility 100 Index", "EURUSD"] 

# Trading Parameters
# FIX: Increased volume from 0.01 to 1.0 (minimum required for Volatility 100 Index)
TRADE_VOLUME = 1.0  
MAX_TICKS = 60       # Number of ticks to fetch before stopping (Increased for more trades)

# Data Storage
tick_data_list = []

# Global placeholder for the logged-in account ID
MT5_LOGIN = None 

# ----------------------------------------------------------------------
# 3. Trading Function (Sends Order to MT5)
# ----------------------------------------------------------------------

def place_trade(symbol, trade_type, volume):
    """
    Submits an order to the connected MT5 terminal.
    
    :param symbol: Trading symbol (e.g., "Volatility 100 Index")
    :param trade_type: mt5.BUY or mt5.SELL
    :param volume: Lot size
    :return: True if the order was sent successfully, False otherwise.
    """
    
    # 1. Check if the symbol is available and selected
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f"!!! Error: Symbol {symbol} not found or MT5 not connected.")
        return False
        
    if not symbol_info.visible:
        print(f"!!! Error: Symbol {symbol} is not visible in MT5 Market Watch. Attempting to select it.")
        if not mt5.symbol_select(symbol, True):
            print(f"!!! Error: Failed to select {symbol}. Cannot trade.")
            return False
            
    # CRITICAL CHECK 1: Ensure the symbol allows full trading (not disabled or close-only)
    if symbol_info.trade_mode != mt5.SYMBOL_TRADE_MODE_FULL:
        print(f"!!! Error: Symbol {symbol} is not available for trading (Trade Mode: {symbol_info.trade_mode}).")
        return False

    # CHECK VOLUME LIMITS (Added for diagnosis of 10014 error)
    print(f"   [Symbol Info] Min Vol: {symbol_info.volume_min}, Max Vol: {symbol_info.volume_max}, Step: {symbol_info.volume_step}")
    if volume < symbol_info.volume_min or volume > symbol_info.volume_max:
         print(f"!!! Error: Requested volume {volume} is outside the allowed range ({symbol_info.volume_min}-{symbol_info.volume_max}).")
         return False
            
    # 2. Get the latest tick/price information
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        print(f"!!! Error: Could not retrieve a tick for {symbol}. Cannot trade.")
        return False
        
    # Determine price and type
    if trade_type == mt5.ORDER_TYPE_BUY:
        price = tick.ask
        action = mt5.TRADE_ACTION_DEAL 
        order_type = mt5.ORDER_TYPE_BUY
    else: # SELL
        price = tick.bid
        action = mt5.TRADE_ACTION_DEAL
        order_type = mt5.ORDER_TYPE_SELL
        
    # CRITICAL CHECK 2: Ensure a valid price was obtained
    if price is None or price <= 0:
        print(f"!!! Error: Invalid price ({price}) retrieved for {symbol}. Cannot trade.")
        return False

    # 3. Prepare the trade request
    request = {
        "action": action,
        "symbol": symbol,
        "volume": volume,
        "type": order_type,
        "price": price,
        "deviation": 20, # Acceptable price deviation from current price (in points)
        "magic": 202409, # Custom ID for the bot (optional)
        "comment": "Auto-Connect Bot",
        "type_time": mt5.ORDER_TIME_GTC, # Good 'til cancelled
        "type_filling": mt5.ORDER_FILLING_FOK, # Fill or Kill
    }
    
    # 4. Send the order
    result = mt5.order_send(request)
    
    # *** CRITICAL FIX: Check if order_send failed immediately and returned None ***
    if result is None:
        print(f"!!! CRITICAL: mt5.order_send() returned None. ")
        print(f"!!! This usually means the MT5 terminal connection was interrupted or blocked.")
        print(f"!!! Please verify the MT5 terminal is running, logged in, and NOT busy with pop-ups.")
        return False
    # **************************

    print(f"\n--- TRADING SIGNAL for {symbol} ({'BUY' if trade_type == mt5.ORDER_TYPE_BUY else 'SELL'}) ---")
    
    # 5. Check the result code
    if result.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"!!! Order Send Failed. Code: {result.retcode}")
        # Use mt5.last_error() for further diagnosis if retcode is not 10009 (Request is completed)
        print(f"   Last MT5 Error (before request): {mt5.last_error()}")
        print(f"   Result Comment: {result.comment}")
        return False
    else:
        print(f"*** TRADE EXECUTED ***")
        print(f"   Order: {result.order}")
        print(f"   Ticket: {result.deal}")
        print(f"   Price: {result.price}")
        print("---------------------------------------")
        return True

# ----------------------------------------------------------------------
# 4. Main Script Execution
# ----------------------------------------------------------------------

if __name__ == "__main__":
    
    # 1. Initialize MT5 Connection (Auto-Connect to running terminal)
    if not mt5.initialize():
        print(f"!!! MT5 Initialization FAILED. Check if terminal is running: {mt5.last_error()}")
        exit()
    
    # 2. Check Login Status
    account_info = mt5.account_info()
    if account_info is None:
        print("!!! FAILED TO CONNECT to a logged-in account.")
        print("!!! Please ensure your MT5 terminal is open and logged into your Deriv account.")
        mt5.shutdown()
        exit()

    # Login is successful via the running terminal
    global MT5_LOGIN
    MT5_LOGIN = account_info.login
    
    print(f"MT5 initialized and connected automatically.")
    print(f"Logged in to account: {MT5_LOGIN} | Server: {mt5.terminal_info().name}")

    # 3. Main Loop for Ticks and Trading
    trades_executed = 0
    start_time = time.time()
    
    print(f"\nStarting live tick fetch for {', '.join(SYMBOLS)}. Max Ticks: {MAX_TICKS}")

    while len(tick_data_list) < MAX_TICKS:
        
        for symbol in SYMBOLS:
            # Get the latest tick information
            tick = mt5.symbol_info_tick(symbol)
            
            # Check for tick validity and uniqueness
            is_new_tick = True
            if tick and tick.bid > 0 and tick_data_list and tick.time == tick_data_list[-1]['epoch']:
                is_new_tick = False
                
            if tick and tick.bid > 0 and is_new_tick:
                
                # Extract and format data
                dt_object = datetime.fromtimestamp(tick.time)
                
                # Store data
                tick_data_list.append({
                    'time': dt_object.strftime('%Y-%m-%d %H:%M:%S.%f'),
                    'symbol': symbol,
                    'bid': tick.bid,
                    'ask': tick.ask,
                    'epoch': tick.time
                })

                # Print the latest tick
                print(f"TICK {len(tick_data_list):02d}/{MAX_TICKS}: "
                      f"[{dt_object.time()}] {symbol} | BID: {tick.bid:0.5f} | ASK: {tick.ask:0.5f}")

                # --- RANDOM TRADING STRATEGY (DEMO) ---
                # Execute a random trade on Volatility 100 Index every 5 ticks (max 4 trades)
                if symbol == "Volatility 100 Index" and trades_executed < 4:
                    if len(tick_data_list) % 5 == 0:
                        
                        # Determine trade direction randomly
                        trade_direction = random.choice([mt5.ORDER_TYPE_BUY, mt5.ORDER_TYPE_SELL])
                        
                        print(f"** RANDOM TRADE TRIGGERED: {'BUY' if trade_direction == mt5.ORDER_TYPE_BUY else 'SELL'} for {symbol} **")
                        
                        if place_trade(symbol, trade_direction, TRADE_VOLUME):
                            trades_executed += 1

        # Small delay to prevent overwhelming the MT5 terminal
        time.sleep(0.1) 
        
    # 4. Clean up
    print("\nMT5 connection shutting down.")
    mt5.shutdown()

    # 5. Display the Collected Data
    if tick_data_list:
        df = pd.DataFrame(tick_data_list)
        df['time'] = pd.to_datetime(df['time'])
        df = df.set_index('time')
        
        print("\n--- Collected Ticks DataFrame ---")
        print(df.drop(columns=['epoch']))
        
    else:
        print("\nNo tick data was collected.")

MT5 initialized and connected automatically.
Logged in to account: 40866995 | Server: MetaTrader 5

Starting live tick fetch for Volatility 100 Index, EURUSD. Max Ticks: 60
TICK 01/60: [22:35:14] Volatility 100 Index | BID: 871.45000 | ASK: 871.70000
TICK 02/60: [22:35:03] EURUSD | BID: 1.15273 | ASK: 1.15284
TICK 03/60: [22:35:14] Volatility 100 Index | BID: 871.45000 | ASK: 871.70000
TICK 04/60: [22:35:03] EURUSD | BID: 1.15273 | ASK: 1.15284
TICK 05/60: [22:35:14] Volatility 100 Index | BID: 871.45000 | ASK: 871.70000
** RANDOM TRADE TRIGGERED: SELL for Volatility 100 Index **
   [Symbol Info] Min Vol: 1.0, Max Vol: 220.0, Step: 0.01

--- TRADING SIGNAL for Volatility 100 Index (SELL) ---
*** TRADE EXECUTED ***
   Order: 4439100919
   Ticket: 4431299184
   Price: 871.49
---------------------------------------
TICK 06/60: [22:35:03] EURUSD | BID: 1.15273 | ASK: 1.15284
TICK 07/60: [22:35:16] Volatility 100 Index | BID: 871.49000 | ASK: 871.74000
TICK 08/60: [22:35:03] EURUSD | BID: 1

In [ ]:
# MT5 conncet function - BEST
def connect_mt5(login=None, password=None, server=None) -> bool:
    #Initializes and connects to the MT5 terminal.
    if not mt5.initialize():
        print(f"MT5 initialization failed. Error code: {mt5.last_error()}")
        return False

    if login and password and server:
        # Try to log in if credentials are provided
        authorized = mt5.login(login, password=password, server=server)
        if not authorized:
            print(f"Login failed: {mt5.last_error()}")
            mt5.shutdown()
            return False
            
    print(f"Successfully connected to MT5 terminal.")
    account_info = mt5.account_info()
    if account_info:
        print(f"Account: {account_info.login}")
    return True